
# 🤖 Aula 9: Criando um Chatbot Totalmente Independente e Autônomo

Neste notebook, vamos construir um chatbot que aprende sozinho, busca informações externas e toma decisões sem intervenção humana.

## 📌 O que faremos:
1. **Entender como funciona um chatbot autônomo e autoaprendizagem**.
2. **Fundamentação matemática**: Aprendizado por reforço e tomada de decisões independentes.
3. **Implementar um chatbot que aprende sozinho e revisa suas respostas automaticamente**.
4. **Conectar o chatbot a fontes externas de informação para buscar conhecimento dinâmico**.
5. **Criar um fluxo de automação completo para autoaprendizado**.

---

## 📖 Como um Chatbot Pode Ser Autônomo?

Um chatbot autônomo não depende de supervisão direta. Ele pode:

1. **Coletar dados**: Busca informações em APIs externas e bancos de dados.
2. **Aprimorar suas respostas**: Usa aprendizado por reforço para melhorar suas interações.
3. **Tomar decisões sozinho**: Escolhe a melhor resposta com base em otimização matemática.

### **Matemática do Aprendizado Contínuo**

Podemos modelar o aprendizado contínuo com **Reinforcement Learning**. Seja \( Q(s, a) \) a função de valor para um chatbot que escolhe uma resposta \( a \) no estado \( s \):

\(
Q(s, a) = Q(s, a) + lpha \cdot (r + \gamma \max_{a'} Q(s', a') - Q(s, a))
\)

onde:
- \( lpha \) é a taxa de aprendizado.
- \( r \) é a recompensa obtida para a resposta dada.
- \( \gamma \) é o fator de desconto (incentiva aprendizado de longo prazo).
- \( Q(s, a) \) representa a qualidade esperada da ação no estado.

Esse modelo permite que o chatbot **melhore progressivamente suas respostas**.


In [ ]:

# Instale a biblioteca OpenAI e Requests se ainda não estiverem instaladas
!pip install openai requests json


In [ ]:

import openai
import requests
import json

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"


In [ ]:

# Criando um histórico para aprendizado contínuo
historico = []

def chatbot_aprendizado(pergunta):
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": pergunta}]
    )["choices"][0]["message"]["content"]
    
    historico.append({"pergunta": pergunta, "resposta": resposta})
    
    with open("historico.json", "w") as f:
        json.dump(historico, f, indent=4)
    
    return resposta

# Testando o chatbot
print(chatbot_aprendizado("O que é aprendizado de máquina?"))


In [ ]:

# Instale as bibliotecas necessárias
!pip install openai faiss-cpu numpy requests json

import openai
import numpy as np
import faiss
import requests
import json

# Defina sua chave de API aqui (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"

# Criar banco de memória vetorial com FAISS para aprendizado contínuo
dimensao = 1536  # Tamanho dos embeddings da OpenAI
indice_memoria = faiss.IndexFlatL2(dimensao)  # Banco de dados vetorial
memorias = []  # Lista de perguntas e respostas armazenadas

# Função para obter embedding de um texto
def obter_embedding(texto):
    resposta = openai.Embedding.create(input=texto, model="text-embedding-ada-002")
    return np.array(resposta["data"][0]["embedding"], dtype=np.float32)

# Função para armazenar conhecimento aprendido
def armazenar_memoria(pergunta, resposta):
    vetor = obter_embedding(pergunta)
    memorias.append((pergunta, resposta))
    indice_memoria.add(np.array([vetor]))

# Função para recuperar a resposta mais relevante
def recuperar_resposta(pergunta):
    if len(memorias) == 0:
        return None
    
    vetor = obter_embedding(pergunta)
    D, I = indice_memoria.search(np.array([vetor]), k=1)
    return memorias[I[0][0]][1] if D[0][0] < 0.5 else None

# Função de aprendizado contínuo
def chatbot_autonomo(pergunta):
    resposta_existente = recuperar_resposta(pergunta)
    if resposta_existente:
        return resposta_existente
    
    historico = [{"role": "system", "content": "Você é um chatbot autônomo que aprende sozinho."},
                 {"role": "user", "content": pergunta}]
    
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )["choices"][0]["message"]["content"]

    armazenar_memoria(pergunta, resposta)
    
    return resposta

# Função para buscar dados externos (Exemplo: Clima)
def buscar_clima(cidade):
    api_key = "SUA_CHAVE_DE_CLIMA"
    url = f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={cidade}"
    resposta = requests.get(url).json()
    return f"A temperatura em {cidade} é {resposta['current']['temp_c']}°C."

# Teste de aprendizado contínuo
print(chatbot_autonomo("O que é aprendizado profundo?"))
print(chatbot_autonomo("Como a IA pode melhorar a eficiência energética?"))

# Teste de busca de informações externas
print(buscar_clima("São Paulo"))


In [ ]:

# Criando um chatbot que decide o que fazer com base na pergunta do usuário
def chatbot_autonomo(pergunta):
    if "clima" in pergunta.lower():
        cidade = pergunta.split("em")[-1].strip()
        return buscar_clima(cidade)
    elif "cotação" in pergunta.lower():
        return "Buscando cotação de mercado... (exemplo de integração financeira)"
    else:
        return chatbot_aprendizado(pergunta)

# Testando chatbot autônomo
print(chatbot_autonomo("Como está o clima em Rio de Janeiro?"))
print(chatbot_autonomo("Qual a cotação do dólar hoje?"))


In [ ]:

# Criando um chatbot que monitora dados e envia alertas automáticos
def monitorar_dados():
    temperatura = buscar_clima("São Paulo")
    if "30°C" in temperatura:
        return "Alerta! Está muito quente, beba bastante água!"
    else:
        return "Clima está normal."

# Testando monitoramento
print(monitorar_dados())



## 🎯 Desafio para os alunos!
1. Criar um chatbot que aprende sozinho e toma decisões baseadas em dados reais.
2. Integrar o chatbot com pelo menos uma API externa.
3. Criar um fluxo de automação completo baseado nas respostas do chatbot.

---
